In [1]:
import time
import numpy as np
import math
import numpy as np
from itertools import combinations
import os
import random
from collections import defaultdict

In [2]:
def load_graph(file):
    '''Loads a graph from a .clq (DIMACS) file'''
    with open(os.path.join('data', file), "r") as f:
        lines = [line for line in f.readlines()]
    lines = [line.strip().split()
             for line in lines if line.strip().split() != []]
    p_line = [line for line in lines if line[0] == 'p'][0]
    e_lines = [line for line in lines if line[0] == 'e']
    n = int(p_line[2])
    print("Number of vertices:", n)
    print("Number of edges:", len(e_lines))
    adjmat = [[0] * n for _ in range(n)]
    for e in e_lines:
        v, w = int(e[1]), int(e[2])
        if v == w:
            print("Loop detected"), v
        if adjmat[v][w]:
            print("Duplicate edge"), v, w
        adjmat[v][w] = adjmat[v][w] = 1
    return (np.array(adjmat), n, len(e_lines))


def enter_matrix():
    R = int(input("Enter the number of rows:"))
    C = int(input("Enter the number of columns:"))
    print("Enter the entries in a single line (separated by space): ")
    # User input of entries in a
    # single line separated by space
    entries = list(map(int, input().split()))
    # For printing the matrix
    matrix = np.array(entries).reshape(R, C)
    print(repr(matrix))
    return(matrix)


def triangles(adj_mat):
    edges = all_edges(adj_mat)
    triangles = []
    for edge in edges:
        for neighbor in neighbors(edge[0], adj_mat):
            if is_edge(neighbor, edge[0], adj_mat) and is_edge(neighbor, edge[1], adj_mat):
                triangles.append(set([neighbor, edge[0], edge[1]]))
    return triangles


def neighbors(nodes, adj_mat):
    '''Takes one or a list of nodes (each one between 1 and N) and returns its neighbors (between 1 and N)'''
    neighbors = set()
    if isinstance(nodes, int):
        nodes = [nodes]
    for node in nodes:
        if node == 0:
            raise Exception("Node can't be 0: nodes are between 1 and N")

        for i in range(adj_mat.shape[0]):
            neighbor = adj_mat[node-1, i]
            if neighbor and (i != node):
                neighbors.add(i+1)
    return neighbors


def is_clique(nodes, adj_mat):
    for (node_i, node_j) in combinations(nodes, 2):
        if not is_edge(node_i, node_j, adj_mat):
            return False
    return True


def find_clique_dumb(node, adj_mat):
    '''Returns one clique to which the vertice "node" belongs'''
    clique = set([node])
    node_neighbors = set(neighbors(node, adj_mat))
    while node_neighbors:
        neighbor = node_neighbors.pop()
        if is_clique(clique.union(set([neighbor])), adj_mat):
            clique.add(neighbor)
    return clique


def is_edge(u, v, adj_mat):
    return adj_mat[u-1, v-1] or adj_mat[v-1, u-1]


def all_edges(adj_mat):
    n = adj_mat.shape[0]
    edges = set()
    for i in range(n):
        for j in range(i+1):
            if adj_mat[i, j]:
                edges.add((i+1, j+1))
    return edges


def is_in_clique(v, clique, adj_mat):
    is_in = True
    if clique is None:
        return False
    for node in clique:
        if not is_edge(v, node, adj_mat):
            is_in = False
    return is_in


def is_solution(nodes_list, adj_mat, v=None):
    "Verifies if the cliques in x up to v are indeed cliques"
    if v is None:
        v = adj_mat.shape[0]
    cliques_dict = cliques_from_list(nodes_list, v)
    for clique_nodes in cliques_dict.values():
        if not is_clique(clique_nodes, adj_mat):
            return False
    return True


def cliques_from_list(nodes_list, v=None):
    '''Takes the list X = [1 1 2 3 3 ... ] of nodes containing the label of their associated clique, and returns a dict of the different cliques'''
    if v is None:
        v = len(nodes_list)
    cliques = dict()
    for i in range(v):
        clique = nodes_list[i]
        if clique == 0:
            continue
        if clique in list(cliques):
            cliques[clique].add(i+1)
        else:
            cliques[clique] = set([i+1])
    return cliques

In [3]:
def backtrack(adj_mat, cliques, v=1,  best=(math.inf, None)):
    n = adj_mat.shape[0]
    # print(cliques)
    # print("CALL: v=" + str(v),
    #       len(set(list(cliques))-set({0})), "cliques=", cliques)
    if v == n:
        if is_solution(cliques, adj_mat):
            if len(set(list(cliques))-set({0})) < best[0]:
                best = (len(set(list(cliques))), cliques_from_list(cliques))
                # print('new best:', best)

    else:
        for i in range(1, v+2):
            cliques[v] = i
            if is_solution(cliques, adj_mat):
                # print(cliques)
                if len(set(list(cliques))-set({0})) < best[0]:
                    # print(len(set(list(cliques))-set({0})), '<', best[0])
                    best = backtrack(adj_mat, cliques, v+1, best)
                    # print('new best:', best[0], best[1])
            cliques[v] = 0
    return best


def main(basePath,file):
    test_graph,_,_ = load_graph(basePath+file)
    start_time = time.time()
    cliques = [0 for x in range(test_graph.shape[0])]
    cliques[0] = 1
    solution = backtrack(test_graph, cliques, 1)
    print("File Name :",file)
    print(solution[0], "cliques:", solution[1])
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#file = '/content/drive/MyDrive/Algorithm Dataset/Dataset only for pranto/brock200_2_b.txt'
#if __name__ == "__main__":
 #   main(file)
basePath = '/content/drive/MyDrive/Algorithm Dataset/random_graphs/clq/'
files = [] 
for i in os.listdir(basePath): 
    if i.endswith('.clq'): 
        files.append(i) 
#print(files)
print(len(files))

if __name__ == "__main__":
  for fileName in files:
    main(basePath,fileName)
    print()


20
Number of vertices: 10
Number of edges: 11
File Name : Gnp10_0.2.clq
5 cliques: {1: {1, 3, 4}, 2: {2, 10}, 3: {5}, 4: {9, 6}, 5: {8, 7}}
--- 0.002525806427001953 seconds ---

Number of vertices: 11
Number of edges: 11
File Name : Gnp11_0.2.clq
6 cliques: {1: {1, 11}, 2: {9, 2, 5}, 3: {10, 3}, 4: {4, 6}, 5: {7}, 6: {8}}
--- 0.019353151321411133 seconds ---

Number of vertices: 13
Number of edges: 11
File Name : Gnp13_0.2.clq
9 cliques: {1: {1, 11, 6}, 2: {2}, 3: {3, 12}, 4: {4}, 5: {5}, 6: {13, 7}, 7: {8}, 8: {9}, 9: {10}}
--- 0.004659891128540039 seconds ---

Number of vertices: 14
Number of edges: 22
File Name : Gnp14_0.2.clq
6 cliques: {1: {1, 11, 12}, 2: {10, 2, 3}, 3: {4, 14}, 4: {5, 6, 7}, 5: {8, 9}, 6: {13}}
--- 0.015328645706176758 seconds ---

Number of vertices: 16
Number of edges: 27
File Name : Gnp16_0.2.clq
8 cliques: {1: {1, 2}, 2: {3, 6}, 3: {4, 7}, 4: {16, 5}, 5: {8, 10, 13}, 6: {9, 12}, 7: {11, 15}, 8: {14}}
--- 11.543941020965576 seconds ---

Number of vertices: 17
